In [ ]:
from transformers import pipeline

# Load a pre-trained financial LLM model, such as FinBERT
classifier = pipeline("sentiment-analysis", model="yiyanghkust/finbert-tone", tokenizer="yiyanghkust/finbert-tone")

# Example input text
financial_text = "The stock price of Tesla rose significantly after the positive earnings report."

# Perform sentiment analysis
result = classifier(financial_text)
print(result)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[{'label': 'Positive', 'score': 0.9999991655349731}]


In [ ]:
import spacy

# Load spaCy model for NER
nlp = spacy.load("en_core_web_sm")

# Example text
text = "Apple Inc. is looking at buying U.K. startup for $1 billion"

# Perform Named Entity Recognition (NER)
doc = nlp(text)
for entity in doc.ents:
    print(entity.text, entity.label_)

Apple Inc. ORG
U.K. GPE
$1 billion MONEY


In [ ]:
# !pip install crewai

In [ ]:
# !pip install 'crewai[tools]'

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from langchain_openai import ChatOpenAI  # Optional, if using OpenAI LLM

# Set your API keys (ensure these are set correctly with actual values)
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
os.environ["SERPER_API_KEY"] = "Serper_API_Key"  # serper.dev API key

# Define your LLM (optional)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Define the Serper search tool for financial data
search_tool = SerperDevTool()

# Define your agents with roles and goals
# 1. Financial Sentiment Analysis Agent
sentiment_analysis_agent = Agent(
    role='Financial Sentiment Analyst',
    goal='Analyze financial sentiment from news, social media, and earnings reports',
    backstory="""You specialize in analyzing the financial market sentiment
    by reviewing news, earnings reports, and social media. You provide
    detailed insights to help with investment decisions.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,  # Specify the LLM for this agent
    tools=[search_tool]  # Use search tool to gather data
)

# 2. Financial Risk Analyst
risk_analyst_agent = Agent(
    role='Financial Risk Analyst',
    goal='Assess financial risk based on earnings reports and market data',
    backstory="""You are an expert in analyzing financial risks.
    You review earnings reports, market trends, and historical data
    to assess potential risks to the financial markets.""",
    verbose=True,
    allow_delegation=False,
    llm=llm  # LLM to help with financial risk analysis
)

# 3. Financial Report Writer
report_writer_agent = Agent(
    role='Financial Report Writer',
    goal='Generate comprehensive financial reports based on sentiment analysis and risk assessment',
    backstory="""You are a financial content writer, specializing in
    converting complex financial data into clear, actionable reports for decision-makers.""",
    verbose=True,
    allow_delegation=True,
    llm=llm  # LLM to help generate a readable financial report
)

# Create tasks for your agents
# Task 1: Sentiment analysis of financial data
task1 = Task(
    description="""Analyze the financial sentiment from the latest news articles,
    social media posts, and earnings reports. Focus on trends that impact
    stock prices and market sentiment.""",
    expected_output="Sentiment analysis summary (positive, neutral, negative) of key financial events",
    agent=sentiment_analysis_agent
)

# Task 2: Financial risk assessment
task2 = Task(
    description="""Assess financial risks based on recent earnings reports,
    regulatory filings, and market trends. Identify any potential risk factors that
    might affect stock performance or overall market stability.""",
    expected_output="Financial risk assessment report summarizing potential market risks",
    agent=risk_analyst_agent
)

# Task 3: Generate a financial report based on the previous tasks
task3 = Task(
    description="""Using the sentiment analysis and risk assessment data,
    generate a comprehensive report. The report should include the key insights,
    risks, and a summary of the market outlook. Ensure the report is clear and actionable
    for financial analysts.""",
    expected_output="Full financial report with analysis of market sentiment and risk assessment",
    agent=report_writer_agent
)

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[sentiment_analysis_agent, risk_analyst_agent, report_writer_agent],
    tasks=[task1, task2, task3],
    verbose=True,
    process=Process.sequential  # Sequential execution
)

# Kick off the tasks
result = crew.kickoff()

# Print result
print("######################")
print(result)